In [291]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
device = T.device("mps")

In [9]:
appa_hours = pd.read_csv("exports/appa_hr_crop_withCreated.csv")

In [34]:
appa_hours = appa_hours[['ZnOR_1', 'ZnOR_2', 'LaFeO3_1', 'LaFeO3_2', 'WO3_1', 'WO3_2',
       'ZnOR_heatR_1', 'ZnOR_heatR_2', 'LaFeO3_heatR_1', 'LaFeO3_heatR_2',
       'WO3_heatR_1', 'WO3_heatR_2', 'ZnOR_heatV_1', 'ZnOR_heatV_2',
       'LaFeO3_heatV_1', 'LaFeO3_heatV_2', 'WO3_heatV_1', 'WO3_heatV_2',
       'Temperature', 'Relative_Humidity', 'Pressure', 'VOC',
       'ZnOR_1_TimeSinceCreated', 'ZnOR_2_TimeSinceCreated',
       'LaFeO3_1_TimeSinceCreated', 'LaFeO3_2_TimeSinceCreated',
       'WO3_1_TimeSinceCreated', 'WO3_2_TimeSinceCreated', 'NO2', 'SO2', 'O3', 'CO']]

In [749]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,median_absolute_error, mean_absolute_error

In [595]:
appa_hours.columns

Index(['ZnOR_1', 'ZnOR_2', 'LaFeO3_1', 'LaFeO3_2', 'WO3_1', 'WO3_2',
       'ZnOR_heatR_1', 'ZnOR_heatR_2', 'LaFeO3_heatR_1', 'LaFeO3_heatR_2',
       'WO3_heatR_1', 'WO3_heatR_2', 'ZnOR_heatV_1', 'ZnOR_heatV_2',
       'LaFeO3_heatV_1', 'LaFeO3_heatV_2', 'WO3_heatV_1', 'WO3_heatV_2',
       'Temperature', 'Relative_Humidity', 'Pressure', 'VOC',
       'ZnOR_1_TimeSinceCreated', 'ZnOR_2_TimeSinceCreated',
       'LaFeO3_1_TimeSinceCreated', 'LaFeO3_2_TimeSinceCreated',
       'WO3_1_TimeSinceCreated', 'WO3_2_TimeSinceCreated'],
      dtype='object')

In [27]:
cols = ['ZnOR_1', 'ZnOR_2', 'LaFeO3_1', 'LaFeO3_2', 'WO3_1', 'WO3_2',
       'ZnOR_heatR_1', 'ZnOR_heatR_2', 'LaFeO3_heatR_1', 'LaFeO3_heatR_2',
       'WO3_heatR_1', 'WO3_heatR_2', 'ZnOR_heatV_1', 'ZnOR_heatV_2',
       'LaFeO3_heatV_1', 'LaFeO3_heatV_2', 'WO3_heatV_1', 'WO3_heatV_2',
       'Temperature', 'Relative_Humidity', 'Pressure', 'VOC',
       'ZnOR_1_TimeSinceCreated', 'ZnOR_2_TimeSinceCreated',
       'LaFeO3_1_TimeSinceCreated', 'LaFeO3_2_TimeSinceCreated',
       'WO3_1_TimeSinceCreated', 'WO3_2_TimeSinceCreated']
outs = ['NO2', 'O3', 'CO']

In [756]:
vsplit = round(len(appa_hours) * 0.2)

X_train = appa_hours.iloc[vsplit:][cols]
X_test = appa_hours.iloc[:vsplit][cols]

Y_train = appa_hours.iloc[vsplit:][outs]
Y_test = appa_hours.iloc[:vsplit][outs]

In [752]:
model = RandomForestRegressor()
model.fit(X_train, Y_train)

/var/folders/gl/2h8nn4ps031g1jysgll17wz00000gn/T/ipykernel_13826/1006863496.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, Y_train)


RandomForestRegressor()

In [754]:
Y_test_pred_r = model.predict(X_test)

print("Random Forest")
print(f"Median Error: {median_absolute_error(Y_test, Y_test_pred_r)} \n Mean error: {mean_absolute_error(Y_test, Y_test_pred_r)}")

Random Forest
Median Error: 10.86 
 Mean error: 12.902733576642337


# Transformers

In [312]:
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

In [11]:
appa_hours.columns

Index(['Time', 'ZnOR_1', 'ZnOR_2', 'LaFeO3_1', 'LaFeO3_2', 'WO3_1', 'WO3_2',
       'ZnOR_heatR_1', 'ZnOR_heatR_2', 'LaFeO3_heatR_1', 'LaFeO3_heatR_2',
       'WO3_heatR_1', 'WO3_heatR_2', 'ZnOR_heatV_1', 'ZnOR_heatV_2',
       'LaFeO3_heatV_1', 'LaFeO3_heatV_2', 'WO3_heatV_1', 'WO3_heatV_2',
       'Temperature', 'Relative_Humidity', 'Pressure', 'VOC', 'PM10', 'NO2',
       'SO2', 'O3', 'CO', 'ZnOR_1_TimeSinceCreated', 'ZnOR_2_TimeSinceCreated',
       'LaFeO3_1_TimeSinceCreated', 'LaFeO3_2_TimeSinceCreated',
       'WO3_1_TimeSinceCreated', 'WO3_2_TimeSinceCreated'],
      dtype='object')

In [13]:
len(['Time', 'ZnOR_1', 'ZnOR_2', 'LaFeO3_1', 'LaFeO3_2', 'WO3_1', 'WO3_2',
       'ZnOR_heatR_1', 'ZnOR_heatR_2', 'LaFeO3_heatR_1', 'LaFeO3_heatR_2',
       'WO3_heatR_1', 'WO3_heatR_2', 'ZnOR_heatV_1', 'ZnOR_heatV_2',
       'LaFeO3_heatV_1', 'LaFeO3_heatV_2', 'WO3_heatV_1', 'WO3_heatV_2',
       'Temperature', 'Relative_Humidity', 'Pressure', 'VOC','ZnOR_1_TimeSinceCreated', 'ZnOR_2_TimeSinceCreated',
       'LaFeO3_1_TimeSinceCreated', 'LaFeO3_2_TimeSinceCreated',
       'WO3_1_TimeSinceCreated', 'WO3_2_TimeSinceCreated'])

29

In [528]:
X[0].shape

(24, 28)

In [451]:
class BasicNet(nn.Module):
    def __init__(self):
        super(BasicNet, self).__init__()
        self.conv1 = Conv2d()
        self.conv2 = Conv2d()
        self.conv3 = Conv2d()

        self.fc1 = nn.Linear(28, 200)
        self.fc2 = nn.Linear(200, 150)
        self.fc3 = nn.Linear(150, 50)
        self.fc4 = nn.Linear(50, 3)

        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = self.dropout2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = self.dropout1(x)
        x = F.relu(x)
        x = self.fc4(x)
        return x

basic_model = BasicNet()
print(basic_model)

BasicNet(
  (fc1): Linear(in_features=28, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=150, bias=True)
  (fc3): Linear(in_features=150, out_features=50, bias=True)
  (fc4): Linear(in_features=50, out_features=3, bias=True)
  (dropout1): Dropout(p=0.25, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
)


In [445]:
basic_model(torch.rand((1,28)))

tensor([[-0.1110,  0.0374,  0.1223]], grad_fn=<AddmmBackward0>)

In [523]:
X = appa_hours[['ZnOR_1', 'ZnOR_2', 'LaFeO3_1', 'LaFeO3_2', 'WO3_1', 'WO3_2',
       'ZnOR_heatR_1', 'ZnOR_heatR_2', 'LaFeO3_heatR_1', 'LaFeO3_heatR_2',
       'WO3_heatR_1', 'WO3_heatR_2', 'ZnOR_heatV_1', 'ZnOR_heatV_2',
       'LaFeO3_heatV_1', 'LaFeO3_heatV_2', 'WO3_heatV_1', 'WO3_heatV_2',
       'Temperature', 'Relative_Humidity', 'Pressure', 'VOC',
       'ZnOR_1_TimeSinceCreated', 'ZnOR_2_TimeSinceCreated',
       'LaFeO3_1_TimeSinceCreated', 'LaFeO3_2_TimeSinceCreated',
       'WO3_1_TimeSinceCreated', 'WO3_2_TimeSinceCreated']].to_numpy()
Y = appa_hours[['NO2', 'O3', 'CO']].to_numpy()

In [524]:
scalerX = preprocessing.StandardScaler().fit(X)
X = scalerX.transform(X)
scalerY = preprocessing.StandardScaler().fit(Y)
Y = scalerY.transform(Y)

In [525]:
X = np.array([x[0] for x in np.lib.stride_tricks.sliding_window_view(X, (24,28))])
Y = Y[X.shape[1]-1:]
(X.shape,Y.shape)

((13678, 24, 28), (13678, 3))

In [526]:
data = []
for i in range(len(X)):
    data.append([X[i], Y[i]])

In [527]:
vsplit = round(len(data) * 0.2)

train_dl = DataLoader(data[vsplit:], batch_size=32, shuffle=True)
test_dl = DataLoader(data[:vsplit], batch_size=32, shuffle=True)

In [413]:
bm = None
def train_model(train_dl, model):
    global bm
    best_loss = float("inf")
    # define the optimization
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(basic_model.parameters(), lr=0.01)
    # enumerate epochs
    for epoch in range(100):
        for i, (inputs, targets) in enumerate(train_dl):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            for i, inp in enumerate(inputs):
                yhat = model(inp.float())
                # calculate loss
                loss = criterion(yhat, targets[i].float())
                # credit assignment
                loss.backward()
            # update model weights
            optimizer.step()
        if best_loss > loss:
            bm = copy.deepcopy(model)
            best_loss = loss
        print(f"epoch {epoch} done, loss {loss}")

In [323]:
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = model(inputs.float())
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = np.vstack(predictions), np.vstack(actuals)
    # calculate accuracy
    acc = (actuals, predictions)
    return acc

In [187]:
import copy

In [452]:
train_model(train_dl, basic_model)

epoch 0 done, loss 0.1954377144575119
epoch 1 done, loss 0.10638030618429184
epoch 2 done, loss 0.8884353041648865
epoch 3 done, loss 0.6040071845054626
epoch 4 done, loss 0.315480500459671
epoch 5 done, loss 0.0491899810731411
epoch 6 done, loss 0.037178680300712585
epoch 7 done, loss 0.022976979613304138
epoch 8 done, loss 0.3955971896648407
epoch 9 done, loss 0.1619979590177536
epoch 10 done, loss 0.030794380232691765
epoch 11 done, loss 0.285727322101593
epoch 12 done, loss 0.3879714012145996
epoch 13 done, loss 0.029069319367408752
epoch 14 done, loss 0.11306282877922058
epoch 15 done, loss 0.13033254444599152
epoch 16 done, loss 0.37385740876197815
epoch 17 done, loss 0.1087261214852333
epoch 18 done, loss 0.08239373564720154
epoch 19 done, loss 0.09780099242925644
epoch 20 done, loss 0.17509759962558746
epoch 21 done, loss 0.3950960636138916
epoch 22 done, loss 0.3915615975856781
epoch 23 done, loss 0.15846747159957886
epoch 24 done, loss 0.009693458676338196
epoch 25 done, loss

In [338]:
basic_model(torch.rand((1,28)))

tensor([[ 0.2144,  0.1129, -0.0328]], grad_fn=<AddmmBackward0>)

In [458]:
from sklearn.metrics import mean_squared_error, median_absolute_error, mean_absolute_error
evals = evaluate_model(test_dl, bm)

median_absolute_error(scalerY.inverse_transform(evals[1]),  scalerY.inverse_transform(evals[0]))

16.295283989376493

In [459]:
def extract_windows(array, clearing_time_index, max_time, sub_window_size):
    examples = []
    start = clearing_time_index + 1 - sub_window_size + 1
    
    for i in range(max_time+1):
        example = array[start+i:start+sub_window_size+i]
        examples.append(np.expand_dims(example, 0))
    
    return np.vstack(examples)